<a href="https://colab.research.google.com/github/PDNow-Research/PDNow/blob/main/HandPD/NeuralNets/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We are going to take text-based features and apply a simple, fully-connected neural network to them. Atually, let's use images - that's what the tutorial does. Let's only consider Meander for the time being.

In [182]:
# General
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import PIL
from PIL import Image
import pandas as pd
import os
import math

# Image Reading
import cv2
import glob

# Other ML Preprocessing
from sklearn.model_selection import train_test_split

# Torch General
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn as nn # loss functions, neural network type (convolutional, linear, etc.)
import torch.optim as optim # optimization functions (sgd)
import torch.nn.functional as F # functions without parameters - activation functions (Relu, etc.) (also included in nn package, could use, but functional package is "better")
from torch.utils.data import DataLoader
import torchvision.datasets as datasets # torch has a LOT LOT LOT of standard datasets (ImageNet, MNIST, etc.)
import torchvision.transforms as transforms # transformations for dataset
from torch.utils.data.sampler import SubsetRandomSampler # PyTorch train test split

Now to create the fully connected network.

In [162]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes): # input-size = 611568 (size of our images, pixel number), num_classes = 2 (PD/no PD)
    super(NN, self).__init__() # initializes the NN class that we're defining
    self.fc1 = nn.Linear(input_size, 50) # 50 nodes
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x): # run on some input x, which is the images which we run through fc1 and fc2 layers created above (and add the reLU activation function it between)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

### Quick Test

What the model should output it something of shape [264 (140 + 124), 2]. For each image, it should predict the probability of it being in class 1 or 2 and return both of those.

In [163]:
model = NN(611568, 2) 

# 124 patient images
# 140 control images 
x = torch.randn(264, 611568) 

print (model(x).shape)

torch.Size([264, 2])


## Set Device + Init Hyperparams

In [164]:
# Device set 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # google colab provides cuda gpu
print (device)

cuda


In [177]:
# Hyperparams
input_size = 611568 # this 611568 comes from the size of each image (maxSize of an image). They have all been padded to be of this size.
num_classes = 2

# tunables
test_size = 0.2
learning_rate = 0.001
batch_size = 64
num_epochs = 1
seed = 0 # just random state to start at

## Load Data

### Preprocessing

In [166]:
# Load Data (possible to do it with Google Drive rather than uploading from computer?).
# Uploaded can be nice tho... especially because it automatically goes to dictionary... that can be fed into NN, with file names as indexes - quite nice

""" from google.colab import files
uploaded = files.upload()"""

# %cd "/content/drive/"My Drive"/Data/Images/Meander/"
# X_train = np.load('HealthyMeander/HealthyMeander/')


' from google.colab import files\nuploaded = files.upload()'

In [167]:
# Healthy: Class 0 . PD: Class 1

def extract_images(path, c): # path of data, class of data
  filename_arr = []
  X_arr = []

  for file in glob.glob(path):
    filename_arr.append(file) # filenames, not going to use them for now. Might need them later.
    x = cv2.imread(file)
    X_arr.append(x)
  
  y_arr = [c] * len(X_arr)

  return X_arr, y_arr

# X_arr is a list of 3D arrays representing images

In [168]:
# possibly try different pad modes for better NN results? (instead of 'constant')
def pad_images(arr):
  arr = np.copy(arr)
  largestX = 0
  largestY = 0

  def pad_condition(pad, largest, index):
    if (2 * pad != (largest - arr[i].shape[index])):
      pad1 = pad + 1
      pad2 = pad
    else:
      pad1, pad2 = pad, pad
    return pad1, pad2

  for i in arr:
    X = i.shape[0]
    Y = i.shape[1]
    if (X > largestX):
      largestX = X
    if (Y > largestY):
      largestY = Y

  for i in range(len(arr)):
    X_pad = int((largestX - arr[i].shape[0]) /2) # pad equally in both directions, must be int
    Y_pad = int((largestY - arr[i].shape[1]) /2)
    
    # but int floors, so we might get something of a slightly wrong shape (by 1), so...
    X_pad1, X_pad2 = pad_condition(X_pad, largestX, 0)
    Y_pad1, Y_pad2 = pad_condition(Y_pad, largestY, 1)

    arr[i] = np.pad(arr[i], ((X_pad1, X_pad2), (Y_pad1, Y_pad2), (0, 0)), 'constant', constant_values=(0))

  maxSize = largestX * largestY
  
  return arr, maxSize

In [169]:
"""
X_meah, y_meah = extract_images("/content/drive/MyDrive/Data/Images/Meander/HealthyMeander/HealthyMeander/*.*",0)
X_meap, y_meap = extract_images("/content/drive/MyDrive/Data/Images/Meander/PatientMeander/PatientMeander/*.*",1)

X_all = X_meah + X_meap
X_all, maxSize = pad_images(X_all)

y_all = y_meah + y_meap"""

'\nX_meah, y_meah = extract_images("/content/drive/MyDrive/Data/Images/Meander/HealthyMeander/HealthyMeander/*.*",0)\nX_meap, y_meap = extract_images("/content/drive/MyDrive/Data/Images/Meander/PatientMeander/PatientMeander/*.*",1)\n\nX_all = X_meah + X_meap\nX_all, maxSize = pad_images(X_all)\n\ny_all = y_meah + y_meap'

In [170]:
class MeanderDataset(Dataset):
  def __init__(self):
    # data loading
    X_meah, y_meah = extract_images("/content/drive/MyDrive/Data/Images/Meander/HealthyMeander/HealthyMeander/*.*",0)
    X_meap, y_meap = extract_images("/content/drive/MyDrive/Data/Images/Meander/PatientMeander/PatientMeander/*.*",1)

    y_all = y_meah + y_meap
    X_all = X_meah + X_meap
    X_all, maxSize = pad_images(X_all) # just padding all of the array's images to be the same size

    X_all = np.stack(X_all, axis=0) # stacking into 4D tensor

    self.n_samples = X_all.shape[0]

    self.x = torch.from_numpy(X_all) # creates tensor from numpy array
    self.y = torch.Tensor(y_all) # y_all is a list, not a numpy array
  
  # support indexing such that dataset[i] can be used to get i-th sample
  def __getitem__(self, index):
        return self.x[index], self.y[index]

  # to return size
  def __len__(self):
    return self.n_samples

In [171]:
dataset = MeanderDataset() # Meander Dataset object

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:792: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, order=order, subok=subok, copy=True)


In [ ]:
first_row = dataset[0]
feature0, label0 = first_row
print(feature0, label0)

# feature0 shape is 744, 822, 3
# label0 is just 0

In [176]:
len(dataset)

264

### DataLoader


In [156]:
# Dataloader to load whole dataset
# Shuffle: shuffle data, good for training
# num_workers: faster loading with multiple subprocesses simultaneously, set to 0 if error occurs when loading

In [180]:
# To perform train test split, we'll use sklearn... https://stackoverflow.com/questions/50544730/how-do-i-split-a-custom-dataset-into-training-and-test-datasets

# generate indices: instead of the actual data we pass in integers instead
train_indices, test_indices, _ , _ = train_test_split(
    range(len(dataset)),
    dataset.y,
    stratify=dataset.y,
    test_size=test_size,
    random_state=seed
)

In [189]:
# train_indices is indices of the training values while test_indices is indicies of the testing values. Let's split our data like such.

# generate subset based on indices
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

In [188]:
len(train_split), len(test_split)

(211, 53)

In [ ]:
train_loader = DataLoader(dataset = train_split, batch_size = batch_size, shuffle = True, num_workers =2)